In [1]:
# Cell 1: Setup environment and dependencies
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv
    
    load_dotenv()
except ImportError:
    pass

# Set up LangSmith environment variables if not already set (optional)
os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

In [2]:
# Cell 2: Initialize Google Gemini model
import getpass
import os


if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [5]:
# Cell 3: Test the model with a simple message
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Explain few-shot prompting briefly, in a maximum of two short paragraphs."),
]

response = model.invoke(messages)
print(response.content)

Few-shot prompting is a technique where a large language model (LLM) is provided with a small number of example input-output pairs directly within the prompt itself. These examples illustrate the desired task, format, or style the model should follow, effectively conditioning its immediate response without altering its underlying weights.

By observing these few instances of a task—be it classification, summarization, or question answering—the LLM learns to infer the underlying pattern or instruction. This allows the model to generalize the demonstrated behavior to a new, unseen query, often resulting in significantly improved performance compared to zero-shot (no examples) or one-shot (a single example) prompting, without requiring extensive fine-tuning.


In [6]:
# Cell 4: Import components for few-shot prompting
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Create output parser
parser = StrOutputParser()

print("Components imported successfully!")

Components imported successfully!


In [7]:
# Cell 5: Create examples for few-shot prompting
# Examples for sentiment analysis
examples = [
    {"input": "I love this movie! It's amazing!", "output": "Positive"},
    {"input": "This food tastes terrible.", "output": "Negative"},
    {"input": "The weather is okay today.", "output": "Neutral"},
    {"input": "I'm so excited about my vacation!", "output": "Positive"},
    {"input": "This service is disappointing.", "output": "Negative"}
]

print("Examples created:")
for i, example in enumerate(examples, 1):
    print(f"{i}. Input: '{example['input']}' → Output: '{example['output']}'")

Examples created:
1. Input: 'I love this movie! It's amazing!' → Output: 'Positive'
2. Input: 'This food tastes terrible.' → Output: 'Negative'
3. Input: 'The weather is okay today.' → Output: 'Neutral'
4. Input: 'I'm so excited about my vacation!' → Output: 'Positive'
5. Input: 'This service is disappointing.' → Output: 'Negative'


In [8]:
# Cell 6: Create the few-shot prompt template
# Define the example template
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Create the few-shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Test the few-shot prompt
formatted_examples = few_shot_prompt.format()
print("Few-shot examples formatted:")
print(formatted_examples)

Few-shot examples formatted:
Human: I love this movie! It's amazing!
AI: Positive
Human: This food tastes terrible.
AI: Negative
Human: The weather is okay today.
AI: Neutral
Human: I'm so excited about my vacation!
AI: Positive
Human: This service is disappointing.
AI: Negative


In [9]:
# Cell 7: Create the complete prompt template with examples
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a sentiment analyzer. Classify the sentiment of the given text as Positive, Negative, or Neutral. Look at these examples:"),
    few_shot_prompt,
    ("human", "Now classify this: {input}")
])

print("Complete prompt template created!")

Complete prompt template created!


In [10]:
# Cell 8: Create and test the few-shot chain
# Create the chain
sentiment_chain = final_prompt | model | parser

# Test with new examples
test_inputs = [
    "This book is incredible!",
    "I hate waiting in long lines.",
    "The meeting was fine.",
    "Best pizza I've ever had!"
]

print("Testing few-shot prompting:")
print("=" * 40)

for text in test_inputs:
    result = sentiment_chain.invoke({"input": text})
    print(f"Input: '{text}'")
    print(f"Output: {result}")
    print("-" * 20)

Testing few-shot prompting:
Input: 'This book is incredible!'
Output: Positive
--------------------
Input: 'I hate waiting in long lines.'
Output: Negative
--------------------
Input: 'The meeting was fine.'
Output: Neutral
--------------------
Input: 'Best pizza I've ever had!'
Output: Positive
--------------------
